In [ ]:
import gradio as gr
import numpy as np
from keras.models import load_model
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing import image
from PIL import Image

def probs(probs):
    dim=[]
    probs = np.copy(probs)
    probs = probs[0]
    for i in range(3):
        ind = np.argmax(probs)
        dim.append(ind)
        probs[ind] = np.min(probs)
    return dim

# Function to preprocess the input image for VGG16
def preprocess_image_vgg16(img):
    img = Image.fromarray(img.astype('uint8'), 'RGB')
    img = img.resize((224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array

# Function to preprocess the input image for Inception
def preprocess_image_inception(img):
    img = Image.fromarray(img.astype('uint8'), 'RGB')
    img = img.resize((299, 299))  # Inception model requires input size of (299, 299)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Inception model expects inputs in the range [0, 1]
    return img_array

def preprocess_image_ResNet(img):
    img = Image.fromarray(img.astype('uint8'), 'RGB')
    img = img.resize((224, 224))  # Inception model requires input size of (299, 299)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Inception model expects inputs in the range [0, 1]
    return img_array


def preprocess_image_Efficientnet(img):
    img = Image.fromarray(img.astype('uint8'), 'RGB')
    img = img.resize((255, 255))  # Efficientnet model requires input size of (255, 255)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array
# Function for model prediction
def classify_image(img, selected_model):
    if img is None:
        return "Please upload an image.","upload image"
    
    if selected_model == "vgg16":
        preprocessed_img = preprocess_image_vgg16(img)
        model = models["vgg16"]
        class_names = ["Ajloun", "Jerash", "Petra", "RomanAmphitheater", "UmmQais", "WadiRum"]



    elif selected_model == "Inception":
        preprocessed_img = preprocess_image_inception(img)
        model = models["Inception"]
        class_names = ["Ajloun", "Jerash", "Petra", "RomanAmphitheater", "UmmQais", "WadiRum"]



    elif selected_model == "Efficientnet":
        preprocessed_img = preprocess_image_Efficientnet(img)
        model = models["Efficientnet"]
        class_names = list(target.values())
        
    elif selected_model == "ResNet":
        preprocessed_img = preprocess_image_ResNet(img)
        model = models["ResNet"]
        class_names = ["Ajloun", "Jerash", "Petra", "RomanAmphitheater", "UmmQais", "WadiRum"]
    else:
        return "Invalid model selection","Select model"

    prediction = model.predict(preprocessed_img)
    class_idx = np.argmax(prediction)
    predicted_label = class_names[class_idx]
    #---------------------------
    dim = probs(prediction)
    result={}
    for i in dim:
        v=class_names[i]
        result[v] =prediction[0][i]
    
    return f"Predicted label: {predicted_label}",result

# Load the pre-trained models
model_paths = {
    "Efficientnet":"model DL3.h5",
    "Inception": "fine_tuned_model_Inception.h5",
    "vgg16": "fine_tuned_model_VGG16.h5",
    "ResNet":"fine_tuned_model_resnet50_new.h5"
}

models = {model_name: load_model(model_path) for model_name, model_path in model_paths.items()}

# Gradio Interface
iface = gr.Interface(
    fn=classify_image,
    inputs=[
        gr.Image(label="Upload Image"),
        gr.Dropdown(
            choices=list(models.keys()),
            label="Select Model"
        )
    ],
    outputs=["text",'label'],
    live=True
)

# Launch the interface
iface.launch()